In [43]:
from VUSentimentLexiconLight import LexiconSent
import re
import json
from sqlite_db import thesisDB, docData, enclose

settings_file = 'D:/thesis/settings - nl.json'
db_file = 'D:/thesis/nl/data.db'

#Load all necessary pipelines
lexicon = LexiconSent('nl')
lexicon.negators |= {'niet','geen','nooit'}
db = thesisDB(db_file)

#POS from frog to lexicon
pos_map = {'ADJ' : 'adj', 'BW'  : 'adv', 'LID' : 'other', 'N' : 'noun', 'SPEC' : 'noun', 
           'TSW' : 'other', 'TW' : 'other', 'VG' : 'other', 'VNW' : 'other', 'VZ' : 'prep', 'WW' : 'verb'}
sentToInt = lambda x: 1 if x == 'positive' else -1 if x == 'negative' else 0

In [44]:
#Get lemmatized corpus from the DB
cursor = db.getTokens(returnColumns=['lemma','pos','docid','paragraph_no'])

corpus = []
currentDoc = 1
currentParagraph = 1
doc = []
paragraph = []

for lemma, pos, docid, paragraph_no in cursor:
    if docid != currentDoc and doc: #We hit a new document!
        doc.append(paragraph)
        corpus.append((currentDoc,doc))
        paragraph = []
        doc = []
        currentDoc = docid
        currentParagraph = paragraph_no
    elif paragraph_no != currentParagraph and paragraph: #New paragraph!
        doc.append(paragraph)
        paragraph = []
        currentParagraph = paragraph_no
    if pos != 'LET':
        paragraph.append((lemma,pos_map[pos]))

doc.append(paragraph)
corpus.append((currentDoc,doc))
cursor.close()

In [45]:
#Pick keywords to signal discussion of EU matters. Keep in mind that these are lemmas!
keywords = ['europees unie','EU','Brussel','europees parlement','europees commissie']

In [46]:
##Reconstruct the corpus to fit in the sentiment polarity tagger.
words = sorted(keywords, key = len) #Count doesn't matter, so neither does order here.
words = [r'\b'+word+r'\b' for word in words] #Append word borders so we don't match substrings
EU_words = re.compile('|'.join(words))
del words

corpus_eu = []
for docid,doc in corpus:
    newdoc = []
    for paragraph in doc:
        if re.search(EU_words,' '.join([lemma for lemma,pos in paragraph])):
            newdoc.append(paragraph)
    corpus_eu.append((docid,newdoc))
del docid,doc,newdoc,EU_words

In [47]:
sentiment = []
for docid,doc in corpus_eu:
    docSents = []
    for paragraph in doc:
        lemmas = [lemma for lemma,pos in paragraph]
        postags = [pos for lemma,pos in paragraph]
        docSents.append(sentToInt(lexicon.getDocSentiment(lemmas,postags,spacyPos = False)))
    sentiment.append((docid,docSents))

0.696915657441973

In [42]:
a = set()
a |= {1}
a

{1}